In [1]:
#!pip install selenium
import pandas as pd
from bs4 import BeautifulSoup
import time
from selenium import webdriver
import re

In [2]:

#PATH = "c:/Program Files (x86)/chromedriver_win32/chromedriver.exe"
PATH = "F:\chromedriver.exe"
options = webdriver.ChromeOptions()
options.add_argument('--lang=en-US')
options.add_argument("start-maximized")


In [3]:
start = time.time()
driver = webdriver.Chrome(PATH, options=options)
root = "https://www.kickstarter.com/projects/sanderbakker/cizor-a-tungsten-carbide-cutting-tool-for-every-day-use"
driver.get(root + '/description')


driver1 = webdriver.Chrome(PATH, options=options)
driver1.get(root + "/creator_bio")


driver2 = webdriver.Chrome(PATH, options=options)
profile = '/'.join(re.sub('projects', 'profile', root).split('/')[:-1])
driver2.get(profile + '/about')


driver3 = webdriver.Chrome(PATH, options=options)
driver3.get(profile + '/created')
time.sleep(5)
desc = BeautifulSoup(driver.page_source, 'html.parser')
creat = BeautifulSoup(driver1.page_source, 'html.parser')
about = BeautifulSoup(driver2.page_source, 'html.parser')
created = BeautifulSoup(driver3.page_source, 'html.parser')
print(time.time() - start)

30.302588939666748


In [4]:
'''
first image, maybe we can extract some information about colors with histograms for example
'''

image = desc.find('div', {'class': 'project-profile__feature_image'}).find('img')['src']

In [5]:
'''
categorical 0 or 1: video or not
'''

has_video = desc.find('video') is not None

In [6]:
## Pledge tiers
'''
numbers of pledg tiers, vector of values, to be converted in US dollars
'''

tiers = desc.find_all("div", {"class": "pledge__info"})

In [7]:
tiers_values = []
for tier in tiers:
    s = str(tier.find("span", {"class": "money"}))
    tiers_values.append(int(re.findall(r'\d+', s)[0]))
    
n_tiers = len(tiers_values)
###########
print(f'n_tiers: {n_tiers}, values:{tiers_values}')

n_tiers: 9, values:[40, 42, 48, 50, 58, 80, 88, 130, 138]


In [8]:
## Story 
'''
text and number of images and gifs
'''

story = desc.find("div", {'id': 'story'}).find_next_sibling()
n_gifs = len(story.find_all('img', {'class': "fit js-lazy-image"}))
n_images = len(story.find_all('img')) - n_gifs
##########
print(f'n_gifs: {n_gifs}, n_images: {n_images}')

# text
story_text = ' '.join([i for i in story.find_all(text=True) if i not in ['\n', ' ']])

n_gifs: 15, n_images: 25


In [9]:
## Risks and challenges
'''
just the text
'''

risks = desc.find("div", {'id': 'risks-and-challenges'})
risks_text = ' '.join([i for i in risks.find_all(text=True) if i not in ['\n', ' ']])

In [10]:
# websites
'''
number of websites
'''

websites = creat.find("ul", {'class':"links list f5 bold"}).find_all('li')
n_websites = len(websites)

In [11]:
# linked facebook account
'''
categorical: 0 or 1, fb account linked or not
'''

fb = creat.find("div", {'class':"facebook py2 border-bottom f5"})
fb_linked = 0 if fb.find("a", {'class': 'popup'}) == None else 1

In [17]:
# collaborators
'''
names and number of collaborators
'''
# number
n_collab = 0
if creat.find("div", {'class': 'pt3 pt7-sm mobile-hide row'}):
    collaborators = creat.find("div", {'class': 'pt3 pt7-sm mobile-hide row'}).findChildren('a')
    n_collab = len(collaborators)
    
# names
collab_names = []
for col in collaborators:
    collab_names.append(re.search(r'/profile/(.*?)/about', str(col)).group(1))   

In [13]:
#creator description
'''
text to analyze
'''
creator_bio = about.find('div', {'class': 'inline-block mb3'}).find_next_siblings()
bio_text = []
for i in creator_bio:
    if i.find(text=True) != None:
        bio_text.append(i.find(text=True))
    
bio_text = ' '.join(bio_text)

In [14]:
# creator websites
'''
number of the creator websites
'''
creator_web = about.find('ul', {'class': 'menu-submenu mb6'}).find_all('li')
n_creator_websites = len(creator_web)

In [15]:
'''
number of previous projects
'''

# all projects from the creator
all_projects = created.find('div', {'id': 'content'}).find_all('div', {'class': 'js-track-project-card'})

for i, card in enumerate(all_projects):
    if card.find('a', {'href':root}) != None:
        prev_projects = all_projects[i+1:]
        break
## previous project titles
n_prev_projects = len([x.find('h3', {'class' :'type-18 light hover-item-text-underline mb1'}) for x in prev_projects])

In [71]:
# data for previous projects
prev_data = [str(x)[:str(x).find('>')] for x in prev_projects]

In [75]:
prev_backers = []
prev_percent_succ = []
n_prev_success = 0
for proj in prev_projects:
    prev_backers.append(int(proj['data-project_backers_count']))
    prev_percent_succ.append(round(float(proj['data-project_percent_raised']), 2))
    if proj['data-project_state'] == 'successful':
        n_prev_success += 1
    

In [80]:
data = pd.DataFrame([[image, has_video, n_tiers, tiers_values, n_images, n_gifs, n_websites,
                      fb_linked, n_collab, collab_names, n_creator_websites, n_prev_projects,
                      n_prev_success, prev_backers, prev_percent_succ ]], 
                    columns=['image', 'has_video', 'n_tiers', 'tiers_values', 'n_images', ' n_gifs', 
                             'n_websites', 'fb_linked', 'n_collab', 'collab_names', 'n_creator_websites',
                             'n_prev_projects', 'n_prev_success', 'prev_backers', 'prev_percent_succ'])
data

,image,has_video,n_tiers,tiers_values,n_images,n_gifs,n_websites,fb_linked,n_collab,collab_names,n_creator_websites,n_prev_projects,n_prev_success,prev_backers,prev_percent_succ
0,https://ksr-ugc.imgix.net/assets/032/273/545/e...,True,9,"[40, 42, 48, 50, 58, 80, 88, 130, 138]",25,15,1,1,4,"[producthype, jellop, backerkit, crowd-ox]",1,5,5,"[2656, 4457, 1679, 57, 83]","[661.51, 993.76, 3893.5, 124.0, 232.87]"
